In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy import optimize
import pandas as pd
import glob
import math
import os

import multiprocessing
from multiprocessing import Pool

In [2]:
# Specifying simulation directory and the directory to save results in
wdir = str(input('Enter simulation directory path: '))

# Specifying a snapshot for temporal analysis
sdir = wdir + 'temporal_analysis/'

Enter simulation directory path: /Users/thepoetoftwilight/Documents/CASSI2020/CASSI2020-Results/m10q_res250/


In [3]:
def plot_snap(info):
    
    snap = info[0]
    redshift = info[1]
    metals = info[2]
    spath_metals = info[3]
    x_min = info[4]
    x_max = info[5]
    y_min = info[6]
    y_max = info[7]
    
    for m in metals:
    
        num_df = pd.read_csv(spath_metals[m] + 'data/num/{0}-num_{1}_data.csv'.format(str(snap), m))
        fit_df = pd.read_csv(spath_metals[m] + 'data/fit/{0}-fit_{1}_data.csv'.format(str(snap), m))
        
        centers = num_df['abundance'].to_list()
        heights = num_df['num_val'].to_list()
        fit = fit_df['fit_val'].to_list()
    
        # Create plots for the numerical PDF and store them
        fig, ax = plt.subplots(figsize = (15, 13))

        if(np.min(centers) > x_min):
            empty_left = [x_min, np.min(centers)]
            zeros = [0, 0]
            ax.plot(empty_left, zeros, lw = 5, color = 'orange')

        if(np.max(centers) < x_max):
            empty_right = [np.max(centers), x_max]
            zeros = [0, 0]
            ax.plot(empty_right, zeros, lw = 5, color = 'orange')

        ax.plot(centers, heights, lw = 5, color = 'orange')

        ax.set_xlabel(
            r'$\left[\frac{{{0}}}{{H}} \right]$'.format(m.title()), 
            fontsize = 36, labelpad = 5)
        ax.set_ylabel(r'$p_{{{0}, X}} \left( \left[ \frac{{{0}}}{{H}} \right] \right)$'.format(m.title()),
             fontsize = 38, labelpad = 10)

        ax.set_title('z = {}'.format(str(round(redshift, 2))), y = 1.04)
        ax.ticklabel_format(axis='both', style='sci', scilimits=(0,0))
        
        ax.set_xlim(xmin = x_min, xmax = x_max)
        ax.set_ylim(ymin = y_min, ymax = y_max)
        
        ax.tick_params(labelsize = 36, pad = 10)
        ax.yaxis.offsetText.set_fontsize(36)
        
        fig.savefig(spath_metals[m] + 'images/num/' + 
                    '{0}-num_{1}.png'.format(str(snap), m.title()))
        plt.close()

        # Create plots for the fitted PDF and store them
        fig, ax = plt.subplots(figsize = (15, 13))

        if(np.min(centers) > x_min):
            empty_left = [x_min, np.min(centers)]
            zeros = [0, 0]
            ax.plot(empty_left, zeros, lw = 5, color = 'orange')

        if(np.max(centers) < x_max):
            empty_right = [np.max(centers), x_max]
            zeros = [0, 0]
            ax.plot(empty_right, zeros, lw = 5, color = 'orange')

        ax.plot(centers, heights, label = 'Simulation', lw = 5, color = 'orange')
        ax.plot(centers, fit, label = 'Analytic Fit', lw = 5, color = 'orange', linestyle = ':')

        ax.set_xlabel(
            '[{}/H]'.format(m.title()), 
            fontsize = 38, labelpad = 5)
        ax.set_ylabel('p([{}/H])'.format(m.title()),
             fontsize = 38, labelpad = 10)

        ax.set_title('z = {}'.format(str(round(redshift, 2))), y = 1.04, fontsize = 38)
        ax.ticklabel_format(axis='both', style='sci', scilimits=(0,0))
        
        ax.tick_params(labelsize = 38, pad = 10)
        ax.yaxis.offsetText.set_fontsize(36)
        
        ax.set_xlim(xmin = x_min, xmax = x_max)
        ax.set_ylim(ymin = y_min, ymax = y_max)
        
        ax.legend(prop={'size': 38}, loc = 'upper right')
        fig.savefig(spath_metals[m] + 'images/fit/' + 
                    '{0}-fit_{1}.png'.format(str(snap), m.title()))
        
        plt.close()
        
        print('Plotted snapshot {0} for {1}'.format(str(snap), m.title()))

In [4]:
# Get all rendered indices

rendered_df = pd.read_csv(sdir + 'rendered_snap_stats.csv')
rendered_indices = rendered_df['snap'].to_list()
redshifts = rendered_df['redshift'].to_list()
num_fits = len(rendered_indices)

# Get rendered metals

metal_df = pd.read_csv((sdir + 'metal_list.csv'))
metals = metal_df['metals'].to_list()

# Create a list of paths for all metals
spath_metals = {}

for m in metals:
    spath_metals[m] = sdir + m + '/'

In [5]:
# Find universal x_lim and y_lim across all snapshots

# Initializing 

x_min = math.inf
x_max = -math.inf
y_min = math.inf
y_max = -math.inf

# Iterating across all metals and all snapshots to find universal plot bounds

for m in metals:
    
    for index in rendered_indices:
        
        num_df = pd.read_csv(spath_metals[m] + 'data/num/{0}-num_{1}_data.csv'.format(str(index), m))
        
        centers = num_df['abundance'].to_list()
        num_vals = num_df['num_val'].to_list()
        
        curr_x_min = np.min(centers)
        curr_x_max = np.max(centers)
        curr_y_min = np.min(num_vals)
        curr_y_max = np.max(num_vals)
        
        if(curr_x_min < x_min):
            x_min = curr_x_min
            
        if(curr_x_max > x_max):
            x_max = curr_x_max
        
        if(curr_y_min < y_min):
            y_min = curr_y_min
            
        if(curr_y_max > y_max):
            y_max = curr_y_max

In [6]:
# Make data arrays for each snapshot
    
if __name__ == '__main__':
    
    n_proc = multiprocessing.cpu_count()
        
    with Pool(processes = n_proc) as pool:
        
        pool.map(plot_snap, [(rendered_indices[i], redshifts[i], metals, spath_metals, x_min, x_max,
                             y_min, y_max) for i in range(0, num_fits)])

Plotted snapshot 190 for C
Plotted snapshot 219 for C
Plotted snapshot 110 for C
Plotted snapshot 141 for C
Plotted snapshot 190 for N
Plotted snapshot 141 for N
Plotted snapshot 110 for N
Plotted snapshot 219 for N
Plotted snapshot 190 for O
Plotted snapshot 110 for O
Plotted snapshot 219 for O
Plotted snapshot 141 for O
Plotted snapshot 190 for Ne
Plotted snapshot 110 for Ne
Plotted snapshot 219 for Ne
Plotted snapshot 141 for Ne
Plotted snapshot 190 for Mg
Plotted snapshot 110 for Mg
Plotted snapshot 219 for Mg
Plotted snapshot 141 for Mg
Plotted snapshot 190 for Si
Plotted snapshot 110 for Si
Plotted snapshot 141 for Si
Plotted snapshot 219 for Si
Plotted snapshot 190 for S
Plotted snapshot 141 for S
Plotted snapshot 110 for S
Plotted snapshot 219 for S
Plotted snapshot 190 for Ca
Plotted snapshot 141 for Ca
Plotted snapshot 110 for Ca
Plotted snapshot 219 for Ca
Plotted snapshot 190 for Fe
Plotted snapshot 141 for Fe
Plotted snapshot 110 for Fe
Plotted snapshot 219 for Fe
Plotted 

Plotted snapshot 198 for O
Plotted snapshot 120 for O
Plotted snapshot 239 for Ne
Plotted snapshot 149 for Ne
Plotted snapshot 198 for Ne
Plotted snapshot 120 for Ne
Plotted snapshot 239 for Mg
Plotted snapshot 149 for Mg
Plotted snapshot 198 for Mg
Plotted snapshot 120 for Mg
Plotted snapshot 239 for Si
Plotted snapshot 149 for Si
Plotted snapshot 198 for Si
Plotted snapshot 120 for Si
Plotted snapshot 239 for S
Plotted snapshot 149 for S
Plotted snapshot 198 for S
Plotted snapshot 120 for S
Plotted snapshot 239 for Ca
Plotted snapshot 149 for Ca
Plotted snapshot 198 for Ca
Plotted snapshot 120 for Ca
Plotted snapshot 239 for Fe
Plotted snapshot 149 for Fe
Plotted snapshot 120 for Fe
Plotted snapshot 198 for Fe
Plotted snapshot 240 for C
Plotted snapshot 150 for C
Plotted snapshot 121 for C
Plotted snapshot 199 for C
Plotted snapshot 240 for N
Plotted snapshot 150 for N
Plotted snapshot 199 for N
Plotted snapshot 121 for N
Plotted snapshot 240 for O
Plotted snapshot 150 for O
Plotted 

Plotted snapshot 128 for Si
Plotted snapshot 177 for Si
Plotted snapshot 206 for Si
Plotted snapshot 247 for Si
Plotted snapshot 128 for S
Plotted snapshot 177 for S
Plotted snapshot 206 for S
Plotted snapshot 247 for S
Plotted snapshot 128 for Ca
Plotted snapshot 177 for Ca
Plotted snapshot 206 for Ca
Plotted snapshot 128 for Fe
Plotted snapshot 247 for Ca
Plotted snapshot 177 for Fe
Plotted snapshot 206 for Fe
Plotted snapshot 129 for C
Plotted snapshot 247 for Fe
Plotted snapshot 178 for C
Plotted snapshot 207 for C
Plotted snapshot 129 for N
Plotted snapshot 248 for C
Plotted snapshot 178 for N
Plotted snapshot 207 for N
Plotted snapshot 129 for O
Plotted snapshot 248 for N
Plotted snapshot 178 for O
Plotted snapshot 207 for O
Plotted snapshot 129 for Ne
Plotted snapshot 248 for O
Plotted snapshot 178 for Ne
Plotted snapshot 207 for Ne
Plotted snapshot 129 for Mg
Plotted snapshot 248 for Ne
Plotted snapshot 178 for Mg
Plotted snapshot 207 for Mg
Plotted snapshot 129 for Si
Plotted 

Plotted snapshot 136 for Fe
Plotted snapshot 214 for Ca
Plotted snapshot 185 for Fe
Plotted snapshot 255 for Ca
Plotted snapshot 137 for C
Plotted snapshot 214 for Fe
Plotted snapshot 186 for C
Plotted snapshot 255 for Fe
Plotted snapshot 137 for N
Plotted snapshot 215 for C
Plotted snapshot 186 for N
Plotted snapshot 256 for C
Plotted snapshot 137 for O
Plotted snapshot 215 for N
Plotted snapshot 186 for O
Plotted snapshot 256 for N
Plotted snapshot 137 for Ne
Plotted snapshot 215 for O
Plotted snapshot 186 for Ne
Plotted snapshot 256 for O
Plotted snapshot 137 for Mg
Plotted snapshot 215 for Ne
Plotted snapshot 186 for Mg
Plotted snapshot 137 for Si
Plotted snapshot 256 for Ne
Plotted snapshot 215 for Mg
Plotted snapshot 186 for Si
Plotted snapshot 137 for S
Plotted snapshot 256 for Mg
Plotted snapshot 215 for Si
Plotted snapshot 186 for S
Plotted snapshot 137 for Ca
Plotted snapshot 256 for Si
Plotted snapshot 215 for S
Plotted snapshot 186 for Ca
Plotted snapshot 137 for Fe
Plotted

Plotted snapshot 298 for N
Plotted snapshot 264 for Ne
Plotted snapshot 298 for O
Plotted snapshot 264 for Mg
Plotted snapshot 298 for Ne
Plotted snapshot 327 for C
Plotted snapshot 356 for C
Plotted snapshot 327 for N
Plotted snapshot 356 for N
Plotted snapshot 327 for O
Plotted snapshot 356 for O
Plotted snapshot 327 for Ne
Plotted snapshot 356 for Ne
Plotted snapshot 264 for Si
Plotted snapshot 327 for Mg
Plotted snapshot 356 for Mg
Plotted snapshot 298 for Mg
Plotted snapshot 264 for S
Plotted snapshot 327 for Si
Plotted snapshot 356 for Si
Plotted snapshot 298 for Si
Plotted snapshot 264 for Ca
Plotted snapshot 327 for S
Plotted snapshot 356 for S
Plotted snapshot 298 for S
Plotted snapshot 264 for Fe
Plotted snapshot 327 for Ca
Plotted snapshot 356 for Ca
Plotted snapshot 298 for Ca
Plotted snapshot 265 for C
Plotted snapshot 327 for Fe
Plotted snapshot 356 for Fe
Plotted snapshot 265 for N
Plotted snapshot 298 for Fe
Plotted snapshot 328 for C
Plotted snapshot 357 for C
Plotted 

...

Plotted snapshot 426 for Si
Plotted snapshot 397 for Si
Plotted snapshot 455 for Fe
Plotted snapshot 484 for C
Plotted snapshot 426 for S
Plotted snapshot 397 for S
Plotted snapshot 456 for C
Plotted snapshot 484 for N
Plotted snapshot 426 for Ca
Plotted snapshot 397 for Ca
Plotted snapshot 456 for N
Plotted snapshot 484 for O
Plotted snapshot 426 for Fe
Plotted snapshot 456 for O
Plotted snapshot 397 for Fe
Plotted snapshot 484 for Ne
Plotted snapshot 427 for C
Plotted snapshot 398 for C
Plotted snapshot 456 for Ne
Plotted snapshot 484 for Mg
Plotted snapshot 427 for N
Plotted snapshot 398 for N
Plotted snapshot 456 for Mg
Plotted snapshot 484 for Si
Plotted snapshot 427 for O
Plotted snapshot 456 for Si
Plotted snapshot 398 for O
Plotted snapshot 484 for S
Plotted snapshot 427 for Ne
Plotted snapshot 398 for Ne
Plotted snapshot 456 for S
Plotted snapshot 484 for Ca
Plotted snapshot 427 for Mg
Plotted snapshot 398 for Mg
Plotted snapshot 456 for Ca
Plotted snapshot 484 for Fe
Plotted 

Plotted snapshot 405 for Ca
Plotted snapshot 434 for Fe
Plotted snapshot 492 for O
Plotted snapshot 464 for O
Plotted snapshot 405 for Fe
Plotted snapshot 435 for C
Plotted snapshot 492 for Ne
Plotted snapshot 464 for Ne
Plotted snapshot 406 for C
Plotted snapshot 435 for N
Plotted snapshot 492 for Mg
Plotted snapshot 406 for N
Plotted snapshot 435 for O
Plotted snapshot 492 for Si
Plotted snapshot 406 for O
Plotted snapshot 435 for Ne
Plotted snapshot 492 for S
Plotted snapshot 406 for Ne
Plotted snapshot 435 for Mg
Plotted snapshot 492 for Ca
Plotted snapshot 406 for Mg
Plotted snapshot 435 for Si
Plotted snapshot 492 for Fe
Plotted snapshot 406 for Si
Plotted snapshot 435 for S
Plotted snapshot 493 for C
Plotted snapshot 406 for S
Plotted snapshot 435 for Ca
Plotted snapshot 493 for N
Plotted snapshot 435 for Fe
Plotted snapshot 406 for Ca
Plotted snapshot 493 for O
Plotted snapshot 436 for C
Plotted snapshot 493 for Ne
Plotted snapshot 436 for N
Plotted snapshot 493 for Mg
Plotted 

Plotted snapshot 587 for Mg
Plotted snapshot 530 for O
Plotted snapshot 559 for N
Plotted snapshot 501 for Mg
Plotted snapshot 587 for Si
Plotted snapshot 530 for Ne
Plotted snapshot 559 for O
Plotted snapshot 587 for S
Plotted snapshot 501 for Si
Plotted snapshot 530 for Mg
Plotted snapshot 559 for Ne
Plotted snapshot 501 for S
Plotted snapshot 587 for Ca
Plotted snapshot 530 for Si
Plotted snapshot 559 for Mg
Plotted snapshot 501 for Ca
Plotted snapshot 587 for Fe
Plotted snapshot 530 for S
Plotted snapshot 559 for Si
Plotted snapshot 501 for Fe
Plotted snapshot 588 for C
Plotted snapshot 530 for Ca
Plotted snapshot 559 for S
Plotted snapshot 502 for C
Plotted snapshot 588 for N
Plotted snapshot 530 for Fe
Plotted snapshot 559 for Ca
Plotted snapshot 502 for N
Plotted snapshot 588 for O
Plotted snapshot 531 for C
Plotted snapshot 559 for Fe
Plotted snapshot 502 for O
Plotted snapshot 588 for Ne
Plotted snapshot 531 for N
Plotted snapshot 560 for C
Plotted snapshot 588 for Mg
Plotted 

Plotted snapshot 509 for S
Plotted snapshot 538 for Mg
Plotted snapshot 595 for Ca
Plotted snapshot 567 for Mg
Plotted snapshot 509 for Ca
Plotted snapshot 538 for Si
Plotted snapshot 567 for Si
Plotted snapshot 595 for Fe
Plotted snapshot 509 for Fe
Plotted snapshot 538 for S
Plotted snapshot 567 for S
Plotted snapshot 596 for C
Plotted snapshot 510 for C
Plotted snapshot 538 for Ca
Plotted snapshot 567 for Ca
Plotted snapshot 596 for N
Plotted snapshot 538 for Fe
Plotted snapshot 567 for Fe
Plotted snapshot 596 for O
Plotted snapshot 539 for C
Plotted snapshot 568 for C
Plotted snapshot 596 for Ne
Plotted snapshot 539 for N
Plotted snapshot 568 for N
Plotted snapshot 596 for Mg
Plotted snapshot 539 for O
Plotted snapshot 568 for O
Plotted snapshot 596 for Si
Plotted snapshot 539 for Ne
Plotted snapshot 568 for Ne
Plotted snapshot 596 for S
Plotted snapshot 539 for Mg
Plotted snapshot 568 for Mg
Plotted snapshot 596 for Ca
Plotted snapshot 539 for Si
Plotted snapshot 596 for Fe
Plotte

Plotted snapshot 576 for S
Plotted snapshot 548 for C
Plotted snapshot 518 for Ca
Plotted snapshot 576 for Ca
Plotted snapshot 548 for N
Plotted snapshot 518 for Fe
Plotted snapshot 576 for Fe
Plotted snapshot 548 for O
Plotted snapshot 519 for C
Plotted snapshot 577 for C
Plotted snapshot 548 for Ne
Plotted snapshot 519 for N
Plotted snapshot 577 for N
Plotted snapshot 548 for Mg
Plotted snapshot 519 for O
Plotted snapshot 577 for O
Plotted snapshot 548 for Si
Plotted snapshot 519 for Ne
Plotted snapshot 548 for S
Plotted snapshot 577 for Ne
Plotted snapshot 519 for Mg
Plotted snapshot 548 for Ca
Plotted snapshot 577 for Mg
Plotted snapshot 519 for Si
Plotted snapshot 548 for Fe
Plotted snapshot 577 for Si
Plotted snapshot 519 for S
Plotted snapshot 577 for S
Plotted snapshot 549 for C
Plotted snapshot 519 for Ca
Plotted snapshot 577 for Ca
Plotted snapshot 549 for N
Plotted snapshot 519 for Fe
Plotted snapshot 577 for Fe
Plotted snapshot 549 for O
Plotted snapshot 520 for C
Plotted s